<a href="https://colab.research.google.com/github/RayNCode/code_collab/blob/main/BestModelWithoutGridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib as plt
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn
sklearn.set_config(display="diagram")
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from itertools import combinations
from sklearn.model_selection import cross_validate

In [ ]:
data_url = "https://raw.githubusercontent.com/RayNCode/code_collab/main/project-3-files"
learn_dataset = pd.read_csv(f"{data_url}/learn_dataset.csv").copy()
learn_dataset_Emp_contract = pd.read_csv(f"{data_url}/learn_dataset_Emp_contract.csv").copy()
learn_dataset_sport = pd.read_csv(f"{data_url}/learn_dataset_sport.csv").copy()
learn_dataset_job = pd.read_csv(f"{data_url}/learn_dataset_job.csv").copy()

code_work_description_map = pd.read_csv(f"{data_url}/code_work_description_map.csv").copy()
city_adm = pd.read_csv(f"{data_url}/city_adm.csv").copy()
code_Club = pd.read_csv(f"{data_url}/code_Club.csv").copy()
departments = pd.read_csv(f"{data_url}/departments.csv").copy()

test_dataset_job = pd.read_csv(f"{data_url}/test_dataset_job.csv").copy()
test_dataset = pd.read_csv(f"{data_url}/test_dataset.csv").copy()
test_dataset_Emp_contract = pd.read_csv(f"{data_url}/test_dataset_Emp_contract.csv").copy()
test_dataset_sport = pd.read_csv(f"{data_url}/test_dataset_sport.csv").copy()

In [ ]:
def preprocess_data_base(learn_dataset_job, work_desc, learn_dataset, dept_code, emp_contract, learn_dataset_sport, code_club, departments):
    # Chargement et fusion des datasets de travail
    merged_df = pd.merge(learn_dataset_job, work_desc, left_on='work_description', right_on='N3', how='left')

    # Conversion des colonnes N2, N1 et N3 en chaînes de caractères
    merged_df['N2'] = merged_df['N2'].astype(str)
    merged_df['N1'] = merged_df['N1'].astype(str)
    merged_df['N3'] = merged_df['N3'].astype(str)

    # Remplir les valeurs manquantes
    merged_df['N2'].fillna(merged_df['N3'].str[:-2], inplace=True)
    merged_df['N1'].fillna(merged_df['N2'].str[:-1], inplace=True)

    # Fusion avec d'autres datasets
    data_2 = pd.merge(learn_dataset, merged_df, on="Id", how="left")
    df_1 = data_2.merge(dept_code, on='insee_code')
    df_2 = df_1.merge(emp_contract, on='Id', how='left')
    df_3 = df_2.merge(learn_dataset_sport, on='Id', how='left')
    df_4 = df_3.merge(code_Club, left_on='Club', right_on='Code', how='left')
    final_df = df_4.merge(departments, on='dep', how='left')

    # Conversion de type pour les colonnes 'Categorie' et 'REG'
    final_df['Categorie'] = final_df['Categorie'].astype(str).astype('object')
    final_df['REG'] = final_df['REG'].astype(str).astype('object')


    # Création et application d'une condition pour filtrer et imputer des valeurs
    condition = (final_df['ACTIVITY_TYPE'] != "TYPE1|1")
    final_df.loc[condition, ['EMOLUMENT', 'Working_hours']] = 0.0
    # Traitement des valeurs manquantes dans les colonnes catégorielles
    categorical_columns = final_df.select_dtypes(include=['object']).columns
    final_df[categorical_columns] = final_df[categorical_columns].fillna("None")


    return final_df

In [ ]:
learn_dataset_base = preprocess_data_base(learn_dataset_job, code_work_description_map, learn_dataset, city_adm, learn_dataset_Emp_contract, learn_dataset_sport, code_Club, departments)

In [ ]:
def preprocess_learn_test(final_df): #ajouter l'ID quelque part
    # Séparer la variable cible si elle est présente
    if 'target' in final_df.columns:
        y = final_df['target'].copy()
        y = np.where(y == 'B', 1, 0)
        X = final_df.drop(['target'], axis='columns')
        X = X.drop(["Id", 'Nom de la commune', 'Nom fédération', 'Nom catégorie', 'Nom du département', 'Code'], axis="columns")
        imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=100, random_state=0)
        columns_to_impute = ['Working_hours', 'EMOLUMENT']
        X[columns_to_impute] = imputer.fit_transform(X[columns_to_impute])
    else:
        X = final_df.copy()
        X_test_id = X['Id'].copy()
        # Suppression des colonnes non nécessaires
        X = X.drop(["Id", 'Nom de la commune', 'Nom fédération', 'Nom catégorie', 'Nom du département', 'Code'], axis="columns")



    if 'target' in final_df.columns:
      return X, y
    else:
      return X, X_test_id

In [ ]:
X, y = preprocess_learn_test(learn_dataset_base)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
preprocessing = make_column_transformer (
    (OneHotEncoder(handle_unknown='ignore'), ['insee_code',"is_student", "OCCUPATION_42", "ACTIVITY_TYPE", "household", "sex", "employer_category", "job_category", "Terms_of_emp",
                       "Eco_sect", "Job_dep", "WORK_CONDITION", "work_description","N3", "N2", "N1", "town_type", "dep", "Emp_contract", "Club", "Categorie", 'REG']),
    (OrdinalEncoder(), ["Highest_degree", "EMPLOYEE_COUNT"]),
    remainder='passthrough')

In [ ]:
logreg = LogisticRegression(solver='lbfgs', max_iter=5000)
LR = make_pipeline(preprocessing, logreg)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LR, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
LR_fit_time = scores['fit_time'].mean()
LR_score_time = scores['score_time'].mean()
LR_accuracy = scores['test_accuracy'].mean()
LR_precision = scores['test_precision_macro'].mean()
LR_recall = scores['test_recall_macro'].mean()
LR_f1 = scores['test_f1_weighted'].mean()
LR_roc = scores['test_roc_auc'].mean()

In [ ]:
gradient_boosting = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
decision_tree = make_pipeline(preprocessing, gradient_boosting)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(decision_tree, X_train, y_train, scoring=scoring, cv=5)
sorted(scores.keys())

dtree_fit_time = scores['fit_time'].mean()
dtree_score_time = scores['score_time'].mean()
dtree_accuracy = scores['test_accuracy'].mean()
dtree_precision = scores['test_precision_macro'].mean()
dtree_recall = scores['test_recall_macro'].mean()
dtree_f1 = scores['test_f1_weighted'].mean()
dtree_roc = scores['test_roc_auc'].mean()


In [ ]:
XGB = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gradient_boosting = make_pipeline(preprocessing, XGB)

scoring = ['accuracy','precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(gradient_boosting, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
xgb_fit_time = scores['fit_time'].mean()
xgb_score_time = scores['score_time'].mean()
xgb_accuracy = scores['test_accuracy'].mean()
xgb_precision = scores['test_precision_macro'].mean()
xgb_recall = scores['test_recall_macro'].mean()
xgb_f1 = scores['test_f1_weighted'].mean()
xgb_roc = scores['test_roc_auc'].mean()

In [ ]:
RF = RandomForestClassifier()
random_forest = make_pipeline(preprocessing, RF)


scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(random_forest, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
forest_fit_time = scores['fit_time'].mean()
forest_score_time = scores['score_time'].mean()
forest_accuracy = scores['test_accuracy'].mean()
forest_precision = scores['test_precision_macro'].mean()
forest_recall = scores['test_recall_macro'].mean()
forest_f1 = scores['test_f1_weighted'].mean()
forest_roc = scores['test_roc_auc'].mean()

In [ ]:
clf = AdaBoostClassifier(n_estimators=50, random_state=42)
adaboost = make_pipeline(preprocessing, clf)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(adaboost, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
adaboost_fit_time = scores['fit_time'].mean()
adaboost_score_time = scores['score_time'].mean()
adaboost_accuracy = scores['test_accuracy'].mean()
adaboost_precision = scores['test_precision_macro'].mean()
adaboost_recall = scores['test_recall_macro'].mean()
adaboost_f1 = scores['test_f1_weighted'].mean()
adaboost_roc = scores['test_roc_auc'].mean()

In [ ]:
models_correlation = pd.DataFrame({
    'Model'       : ['Logistic Regression', 'Gradient Boosting', 'XGBoost', 'Random Forest', 'Adaboost'],
    'Fitting time': [LR_fit_time, dtree_fit_time, xgb_fit_time, forest_fit_time, adaboost_fit_time],
    'Scoring time': [LR_score_time, dtree_score_time, xgb_score_time,forest_score_time, adaboost_score_time],
    'Accuracy'    : [LR_accuracy, dtree_accuracy, xgb_accuracy, forest_accuracy, adaboost_accuracy],
    'Precision'   : [LR_precision, dtree_precision, xgb_precision, forest_precision, adaboost_precision],
    'Recall'      : [LR_recall, dtree_recall, xgb_recall, forest_recall, adaboost_recall],
    'F1_score'    : [LR_f1, dtree_f1, xgb_f1, forest_f1, adaboost_f1],
    'AUC_ROC'     : [LR_roc, dtree_roc, xgb_roc, forest_roc, adaboost_roc],
    }, columns = ['Model', 'Fitting time', 'Scoring time', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'AUC_ROC'])

models_correlation.sort_values(by='Accuracy', ascending=False)

,Model,Fitting time,Scoring time,Accuracy,Precision,Recall,F1_score,AUC_ROC
2,XGBoost,3.120254,0.335858,0.864930,0.859768,0.853224,0.864311,0.944298
3,Random Forest,73.720035,1.151724,0.858379,0.852202,0.847265,0.857881,0.935914
1,Gradient Boosting,12.608822,0.298119,0.829275,0.825040,0.809950,0.827246,0.911467
4,Adaboost,3.990189,0.539284,0.823849,0.818121,0.805488,0.822061,0.910438
0,Logistic Regression,8.954012,0.349314,0.800370,0.790873,0.783690,0.799124,0.881027
